In [ ]:
from sepal_ui import aoi

In [ ]:
aoi_view = aoi.AoiView()
aoi_view

In [ ]:
# check if an aoi have been selected
assert aoi_view.model.name, "the country have not been selected. please validate an AOI"

In [ ]:
# set the dte of the analysis
calib = {
    "start": 2015,
    "end": 2018,
}

valid = {
    "start": 2018,
    "end": 2021,
}

In [ ]:
# get the data from hansen dataset using earthengine
from sepal_ui.scripts import utils as su
from sepal_ui import mapping as sm
import ee

su.init_ee()

# 1 for deforestation on the calibration period
# 2 for deforestation on the validation period
# 3 for the remaining forest at the end of validation
# NoData value is set to 0
dataset = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")
forest_thres = 10

bands = {
    "A": dataset.select("treecover2000"),
    "B": dataset.select("lossyear").unmask(0),
}

calc = "fcc = "
calc += f"(A>{forest_thres})*(B>{calib['start']-2000})*(B<={calib['end']-2000})*1 + "
calc += f"(A>{forest_thres})*(B>{valid['start']-2000})*(B<={valid['end']-2000})*2 + "
calc += f"(A>{forest_thres})*(B==0)*3"

filtered_data = dataset.expression(calc, bands).mask(dataset.select("datamask")).int8()

# create a map to flex
m = sm.SepalMap()
vi = sm.ValueInspector(m=m)
m.add_control(vi)
m.addLayer(
    filtered_data.select("fcc").clip(aoi_view.model.feature_collection),
    {"palette": ["black", "red", "orange", "green"], "min": 0, "max": 3},
    "forest cover change",
)
m.zoom_ee_object(aoi_view.model.feature_collection)

In [ ]:
# name your computation
from component import parameter as pm
from pathlib import Path

name = "test"
result_dir = pm.result_dir / name
result_dir.mkdir(exist_ok=True)

In [ ]:
# download the map
from component import scripts as cs
from matplotlib import pyplot as plt

# create a folder to save the images
fcc_dir = result_dir / "fcc"
fcc_dir.mkdir(exist_ok=True)

# create a grid around the aoi
grid = cs.set_grid(aoi_view.model.gdf)

fig, ax = plt.subplots()
grid.plot(ax=ax, edgecolor="black")
aoi_view.model.gdf.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=2)

In [ ]:
# download all the images
from tqdm.notebook import tqdm
from sepal_ui.scripts import utils as su
import tempfile
from urllib.request import urlretrieve
import zipfile

pbar = tqdm(total=len(grid), unit="image", unit_scale=True)

for i, r in grid.iterrows():
    geometry = grid.filter(items=[i], axis=0)
    ee_geometry = ee.Geometry(r.geometry.__geo_interface__)
    image = filtered_data.select("fcc").clip(aoi_view.model.feature_collection).unmask()
    link = image.getDownloadURL(
        {
            "name": f"fcc_{i}",
            "region": ee_geometry,
            "filePerBand": False,
            "scale": 30,
            "crs": "EPSG:3857",
        }
    )

    with tempfile.NamedTemporaryFile() as f:
        dst = fcc_dir / f"fcc_{i}.tiff"

        if not dst.is_file():
            urlretrieve(link, f.name)
            with zipfile.ZipFile(f.name, "r") as zip_:
                data = zip_.read(zip_.namelist()[0])
                dst.write_bytes(data)

    pbar.update()

In [ ]:
# create a vrt from it
from osgeo import gdal

fcc_file = fcc_dir / "fcc.vrt"
filepaths = [str(f) for f in fcc_dir.glob("*.tiff")]
ds = gdal.BuildVRT(str(fcc_file), filepaths)
ds.FlushCache()

In [ ]:
import multiprocessing as mp
import pkg_resources

import numpy as np
import pandas as pd
from tabulate import tabulate

import riskmapjnr as rmj

ncpu = mp.cpu_count() - 2
results_makemap = rmj.makemap(
    fcc_file=str(fcc_file),
    time_interval=[calib["end"] - calib["start"], valid["end"] - valid["start"]],
    output_dir=str(result_dir),
    clean=False,
    dist_bins=np.arange(0, 1080, step=30),
    win_sizes=np.arange(5, 100, 8),
    ncat=30,
    parallel=True,
    ncpu=ncpu,
    methods=["Equal Interval", "Equal Area"],
    csize=40,
    no_quantity_error=True,
    figsize=(6.4, 4.8),
    dpi=100,
    blk_rows=128,
    verbose=True,
)

In [ ]:
ws_hat = results_makemap["ws_hat"]
m_hat = results_makemap["m_hat"]
dist_thresh = results_makemap["dist_thresh"]
print(f"The distance theshold is {dist_thresh} m.")
print(f"The best moving window size is {ws_hat} pixels.")
print(f"The best slicing algorithm is '{m_hat}'.")

In [ ]:
from importlib import reload

matplotlib = reload(matplotlib)
import rasterio as rio
from rasterio.plot import show

ifile = result_dir / "endval" / f"riskmap_ws{ws_hat}_{m_hat}_ev.tif"

with rio.open(ifile) as src:
    data = src.read(1)

show(data)